In [ ]:
""" Scrape the r/technews subreddit through the Python Reddit API """

In [1]:
import praw
import pandas as pd
import pickle

In [2]:
# import my Reddit API account info saved in separate .py
from reddit_api_acct import ab_client, ab_secret, ab_agent

In [3]:
reddit = praw.Reddit(client_id = ab_client, 
                     client_secret = ab_secret, 
                     user_agent = ab_agent)


Version 7.0.0 of praw is outdated. Version 7.1.4 was released Sunday February 07, 2021.


In [3]:
hot_posts = reddit.subreddit('technews').hot(limit=10)
for post in hot_posts:
    print(post.title)

Texas’ power grid crumples under the cold
Amazon is developing a new digital currency
New type of supply-chain attack hit Apple, Microsoft and 33 other companies
Amazon will build Fire Sticks for its first device manufacturing line in India
TikTok hit with consumer, child safety and privacy complaints in Europe
China starts commissioning maglev trains with speeds up to 600 km/385 miles per hour
Twitch streamer made $16,000 filming himself sleeping and letting viewers disturb him with alarm clocks and fake dog barks
Graphene “nano-origami” creates tiniest microchips ever
SpaceX Starship SN10 launch this week has ‘60% chance of landing’, says Elon Musk
France fines Google 1.1 million euros over hotel rankings practices


In [4]:
posts = []
ml_subreddit = reddit.subreddit('technews')
for post in ml_subreddit.new(limit=None):  # or .hot
    posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created])
posts = pd.DataFrame(posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created'])


In [11]:
posts.shape

(276, 8)

In [ ]:
# even with No Limit, the Reddit API is limited to at max 1K posts

In [5]:
posts.head()

,title,score,id,subreddit,url,num_comments,body,created
0,Jeff Bezos Becomes World’s Richest Person Agai...,582,lm0g8v,technews,https://www.globaltrends.us/2021/02/jeff-bezos...,130,,1.613614e+09
1,France ties Russia’s Sandworm to a multiyear h...,739,lluqyh,technews,https://arstechnica.com/information-technology...,22,,1.613598e+09
2,Reddit removed 6% of content on its platform i...,454,llvcbf,technews,https://www.axios.com/reddit-content-removal-2...,50,,1.613600e+09
3,Google to invest $75 million in coronavirus-hi...,368,llwqq7,technews,https://www.reuters.com/article/us-eu-alphabet...,18,,1.613604e+09
4,"Printed in days, a house: New York firm takes ...",108,lm36bx,technews,https://www.reuters.com/article/us-usa-tech-3d...,6,,1.613621e+09


In [ ]:
####################

In [26]:
sub = ['technews']  # make a list of subreddits you want to scrape the data from

for s in sub:
    subreddit = reddit.subreddit(s)   # Chosing the subreddit


In [27]:
post_dict = {
    "title" : [],
    "score" : [],
    "id" : [],
    "url" : [],
    "num_comms": [],
    "created" : [],
    "body" : []
}
comments_dict = {
    "comment_id" : [],
    "comment_parent_id" : [],
    "comment_body" : [],
    "comment_link_id" : []
}
    

In [28]:
# ran for top posts on 2/17/2021 

#for submission in subreddit.search(query,sort = "top",limit = 1):
for submission in subreddit.top(limit=None):
    post_dict["title"].append(submission.title)
    post_dict["score"].append(submission.score)
    post_dict["id"].append(submission.id)
    post_dict["url"].append(submission.url)
    post_dict["num_comms"].append(submission.num_comments)
    post_dict["created"].append(submission.created)
    post_dict["body"].append(submission.selftext)

    ##### Accessing comments on the post
    submission.comments.replace_more(limit = None)
    for comment in submission.comments.list():
        comments_dict["comment_id"].append(comment.id)
        comments_dict["comment_parent_id"].append(comment.parent_id)
        comments_dict["comment_body"].append(comment.body)
        comments_dict["comment_link_id"].append(comment.link_id)

post_comments = pd.DataFrame(comments_dict)
post_data = pd.DataFrame(post_dict)
  

In [29]:
post_comments.shape

(203847, 4)

In [30]:
post_comments.head()

,comment_id,comment_parent_id,comment_body,comment_link_id
0,fpgu5a7,t3_gdd9d8,[deleted],t3_gdd9d8
1,fpgnsbq,t3_gdd9d8,big props to him for having the huge balls to ...,t3_gdd9d8
2,fpgquur,t3_gdd9d8,Chickenshit is correct. Attempting or even see...,t3_gdd9d8
3,fpgtifa,t3_gdd9d8,When you're in a position of power but still c...,t3_gdd9d8
4,fpgtfiy,t3_gdd9d8,"Nice, I got to interview him a few years ago w...",t3_gdd9d8


In [31]:
post_data.shape

(962, 7)

In [32]:
post_data.head()

,title,score,id,url,num_comms,created,body
0,"Amazon VP Resigns, Calls Company ‘Chickenshit’...",56834,gdd9d8,https://www.vice.com/en_us/article/z3bjpj/amaz...,1760,1.588634e+09,
1,Robinhood plummets back down to a one-star rat...,34487,lakvbj,https://www.theverge.com/2021/2/1/22261178/rob...,705,1.612261e+09,
2,Twitter hides Trump tweet attacking Supreme Co...,29023,jn69ys,https://techcrunch.com/2020/11/02/twitter-trum...,1267,1.604418e+09,
3,Parler CEO says even his lawyers are abandonin...,24305,kuuq6y,https://www.theverge.com/2021/1/10/22223956/pa...,1485,1.610367e+09,
4,Trump blocked by Twitter and Facebook,23597,ks63ds,https://www.bbc.co.uk/news/technology-55569604,667,1.610024e+09,


In [33]:
# pickle posts and comments

with open('post_data_top.pickle', 'wb') as to_write:
    pickle.dump(post_data, to_write)

    
with open('post_comments_top.pickle', 'wb') as to_write:
    pickle.dump(post_comments, to_write)